<a href="https://colab.research.google.com/github/chou-dai/MNIST-Application/blob/feature%2Fmodel/model/MNIST_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [12]:
# --- ニューラルネットワークの定義 ---

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # 畳み込み層（入力数、出力数、カーネルサイズ）
        self.conv1 = nn.Conv2d(1, 6, 5)
        # Maxプーリング層（ウィンドウサイズ、ストライドサイズ）
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        # 全結合層（入力数、出力数）
        self.fc1 = nn.Linear(16*4*4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # 畳み込み層→ReLU関数→プーリング層
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.pool2(torch.relu(self.conv2(x)))
        # 中間特徴量を1次元に変換
        x = x.view(-1, 16*4*4)
        # 全結合層間をReLU関数
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        # ソフトマックス関数
        x = torch.softmax(x, dim=1)
        return x

In [17]:
# --- データの読み込みと前処理 ---

# 画像データをTensor形式に変換し、平均0.5、標準偏差0.5で正規化するためのTransformsパイプライン
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

# 訓練用データの生成
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=2)

# テスト用データの生成
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=2)

In [14]:
# --- 損失関数と最適化アルゴリズムの定義 ---

net = Net()
# クロスエントロピー誤差関数
criterion = nn.CrossEntropyLoss()
# 最適化アルゴリズムは確率的勾配降下法（SGD）
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
# --- 学習の実行 ---

# エポック数10
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        # 勾配を0にリセット
        optimizer.zero_grad()
        outputs = net(inputs)
        # 出力値と正解ラベルの誤差を算出
        loss = criterion(outputs, labels)
        # 誤差逆伝播からパラメータの勾配を算出
        loss.backward()
        # パラメータを最適化
        optimizer.step()

        running_loss += loss.item()
        if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print('Finished Training')
# loss: 1.475

In [16]:
# --- モデルの保存 ---

torch.save(net.state_dict(), 'model')